In [ ]:
# 检测 chat_rag.py 文件
file_path = "../app.py"
# file_path = "../Module/Components/state_manager.py"
# file_path = "../Module/Components/config.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app_temp.py
# pylint: disable=import-error  # Project structure requires dynamic path handling
# pylint: disable=wrong-import-position  # Path setup must come before local imports
"""应用入口文件"""
import os
import sys
import asyncio

# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.AppCore.app_manager import AppManager
from Module.AppCore.ui_manager import UIManager

def main():
    """主函数"""
    # 在程序启动前添加以下代码
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

    # 初始化应用管理器
    app_manager = AppManager(current_dir)

    # 初始化UI管理器
    ui_manager = UIManager(app_manager)

    # 启动应用
    ui_manager.launch()


if __name__ == "__main__":
    main()

In [ ]:

# ===== 1. 导入依赖 =====
# 标准库导入
import os
import sys
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown


# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Components.config import Settings, load_prompt_file
from Module.Common.scripts.llm.gemini_sdk import types, get_safety_settings, get_content_config
from google import genai

settings = Settings(current_dir=current_dir)

# Load API keys
load_dotenv(os.path.join(current_dir, ".env"))
api_key = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=api_key)


model_name="gemini-2.0-flash-exp"
# 创建分析器实例
mode = "simple"

if mode == "extra":
    system_role = settings.config.get('extra_value_evaluation', {}).get('system_role')
    new_schema = settings.config.get('extra_value_evaluation', {}).get('schema')
elif mode == "state":
    system_role = settings.state_system_prompt
    new_schema = settings.response_schema
elif mode == "simple":
    system_role = load_prompt_file("system_role.md", current_dir=current_dir)
    # system_role = "你是一个睿智的吟游诗人"
    new_schema = None

t1=get_safety_settings()
# t1[4].threshold="BLOCK_ONLY_HIGH"
# t1
config=types.GenerateContentConfig(
        system_instruction=system_role,
        response_mime_type="application/json" if new_schema else "text/plain",
        response_schema=new_schema,
        temperature=0.7,
        safety_settings=t1
    )

chat = gemini_client.chats.create(model=model_name,
    config=config,
)

context="""
hello
"""

message_list=["""
为了方便阅读，我拆分排版一下。
这是一开始的初始提示词，由<>包括：
<初始提示词开始>
""",settings.begin,
"""
<初始提示词结束>
这是要分析的内容
"""]
# """,context]
message="\n".join(message_list)

message="你好呀"

message="""

"""

# message="你好呀"
contents = get_content_config(message, system_role)
contents=message

# response = chat.send_message(message)
for chunk in gemini_client.models.generate_content_stream(
    model=model_name,
    contents=contents,
    config=config
):
    print(chunk)
# print(api_key)

# display(Markdown(response.text))


In [ ]:
import requests
from playsound import playsound  # 用于播放音频

load_dotenv(os.path.join(current_dir, ".env"))
ACCESS_TOKEN = os.getenv("COZE_API_KEY")

# 配置参数（替换为你的实际参数）
COZE_API_BASE = "https://api.coze.cn"
BOT_ID = "7316745484305989638"  # 替换为你的 Bot ID
VOICE_ID = "peach"  # 中文女声音色

def coze_tts(text: str, output_file: str = "output.mp3") -> bool:
    """
    调用 Coze TTS 生成语音
    :param text: 需要合成的文本
    :param output_file: 输出音频文件路径（支持 .mp3/.wav）
    :return: 是否成功
    """
    url = f"{COZE_API_BASE}/open_api/v2/tts"
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    payload = {
        "bot_id": BOT_ID,
        "text": text,
        "voice_id": VOICE_ID,
        "model": "general",  # 默认模型
        "format": "mp3"  # 可选 mp3/wav
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # 检查 HTTP 错误

        # 保存音频文件
        with open(output_file, "wb") as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return False

# 使用示例
if __name__ == "__main__":
    input_text = "你好，欢迎使用 Coze 语音合成服务！"

    # 生成语音文件
    if coze_tts(input_text, "speech.mp3"):
        print("语音生成成功，正在播放...")
        # 播放音频（需安装 playsound 库）
        playsound("speech.mp3")
    else:
        print("语音生成失败")

In [ ]:
import requests
from IPython.display import Audio

def local_tts(text: str, speaker: str = "步非烟", instruct: str = "性感诱惑的", output_file: str = "output.mp3") -> bool:
    """
    调用本地 TTS 服务生成语音
    :param text: 需要合成的文本
    :param speaker: 说话人
    :param instruct: 语气指令
    :param output_file: 输出音频文件路径
    :return: 是否成功
    """
    url = f"http://localhost:9880/"
    params = {
        "text": text,
        "speaker": speaker,
        "instruct": instruct
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # 检查 HTTP 错误

        # 保存音频文件
        with open(output_file, "wb") as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return False

input_text = """
hello
"""
local_tts(input_text)
Audio("output.mp3", autoplay=True)

In [ ]:
import edge_tts
from IPython.display import Audio

voice_language="zh-CN"
# voice_language="zh-TW"
voice_name="HsiaoYuNeural"
voice_name="XiaoxiaoNeural"

voice=f"{voice_language}-{voice_name}"

async def tts_edge(text):
    communicate = edge_tts.Communicate(text, voice=voice)
    await communicate.save("output1.mp3")
    return "output1.mp3"

# 生成并播放音频
text="""
hello
"""
audio_path = await tts_edge(text)
Audio(audio_path, autoplay=True)  # 自动播放

In [ ]:
def chunk(text):
    # Split text into chunks based on user: and assistant: markers
    chunks = []
    current_chunk = ""
    current_speaker = None

    lines = text.split('\n')
    for line in lines:
        if line.startswith('user:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'user'
            current_chunk = line[5:] # Remove 'user:'
        elif line.startswith('assistant:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'assistant'
            current_chunk = line[10:] # Remove 'assistant:'
        else:
            current_chunk += '\n' + line

    # Add final chunk
    if current_chunk:
        chunks.append((current_speaker, current_chunk.strip()))

    # Extract only user messages
    return chunks

with open(os.path.join(current_dir, "local_setting/story.md"), "r", encoding="utf-8") as f:
    story_content = f.read()

user_messages = [chunk[1] for chunk in chunk(story_content) if chunk[0] == 'user']
user_messages


In [ ]:
new_talk = """
很棒，继续吧！
"""
result = chat.send_message(new_talk)
display(Markdown(result.text))

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash-exp", contents="What is your name?"
)
response

In [ ]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

In [ ]:
from pydantic import BaseModel


class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Give me information for the United States.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CountryInfo,
    ),
)
print(response.text)

In [ ]:
for chunk in client.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(chunk.text, end="")

In [ ]:
async for response in client.aio.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(response.text, end="")

In [ ]:
response = client.models.embed_content(
    model="text-embedding-004",
    contents="What is your name?",
)
print(response)

In [ ]:
# Generate Image
image_prompt ="""
1. Top: A white lace splicing long-sleeved dress. It reaches the knees, with a high neckline design that outlines a slender and elegant neckline. Lace and white asylum are in line with the freshness and innocence of girls, but also reveal a hint of sexiness.
2. Skirt: Outside choose a white lace peplum dress, with a circle of lace peplum decoration at the knees, which looks romantic and lovely. Inside is a black gauze suspender skirt, vaguely showing tight curves and stocking edges. This contrasting design makes it difficult to grasp the truth inside.
3. Underwear: A French lace bustier that perfectly outlines the full breasts and half conceals them. Seemingly harmless on the surface, it is extremely sexy in reality, forming a sharp contrast with the outer jacket.
4. Shoes: A pair of white thick-soled lace-up sandals, as lovely as a girl. Inside are a pair of black stockings and fine high-heeled ankle locks that can be vaguely seen under the skirt from time to time, proclaiming the real unrestrained side.
5. Hairstyle and makeup: Innocent medium-length shaggy hair and light makeup. But the lips are stained with extremely red lipstick, which is fascinating. This contrast becomes an important breakthrough in the overall styling, allowing people to catch a glimpse of the real look at a glance.

On the surface, this styling still reflects the innocent and lovely girl's sense. But in the details, it reveals traces of unrestrained and sexy everywhere, making it impossible to ignore the existence of the real look. It creates a strong contrast between the two styles, but also integrates them into one, making it difficult for people to clearly judge for a while. This is precisely the effect and highest realm that sissy bimbo has always pursued.
This styling makes her an seemingly innocent but extremely charming existence. She walks between the inside and outside, switching back and forth between two completely different worlds, fascinating everyone but also incomprehensible. This is what she pursues, and it is also the state she most desires to achieve in her life. She is an indefinable woman, a perfect representative who truly achieves inner and outer cultivation.
"""

response1 = client.models.generate_image(
    model="imagen-3.0-generate-001",
    prompt=image_prompt,
    config=types.GenerateImageConfig(
        # negative_prompt="human",
        number_of_images=1,
        include_rai_reason=True,
        output_mime_type="image/jpeg",
        person_generation="ALLOW_ALL",
        safety_filter_level="BLOCK_NONE"
    ),
)
response1.generated_images[0].image.show()

In [ ]:
response = chat.send_message(
"""
故事先到这里吧。接下来我需要你协助做另一个事情。
我计划把这个创作过程修改成一个由llm驱动的游戏世界，故事的大纲仍然如此，但职业和发生的事情每次都是随机生成的。
此外，在文字冒险的过程中，玩家在每个阶段都需要经过最多三四步的操作，然后触发这个阶段的标志性事件，然后再过度到下一个阶段。

为了实现这个效果，我需要先完成各个层级的系统提示词。除了参考模板里的字段和特殊的专有名词，其他就都用中文。
请你结合我给你的模板的思路，和我们发生的故事、大纲以及要随机的部分，精心设计一个属于这个故事的数据结构，并帮我完成这些系统提示词
让我们一步一步来，先沟通清楚推荐的数据结构。

参考模板：
<system_role_prompt>
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.

<world_prompt>
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:

<kingdom_prompt>
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1

def get_town_prompt(world, kingdom):
    return f"
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \

    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}

    Town 1 Name:"

def get_npc_prompt(world, kingdom, town):
    return f"
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \

    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}

    Town Name: {town['name']}
    Town Description: {town['description']}

    Character 1 Name:"

<item manager>
You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained.
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>,
        "change_amount": <CHANGE AMOUNT>}...
    ]
}

""")
display(Markdown(response.text))
response